# Tree sequence workshop

Welcome to the tree sequence workshop. This will give you an overview of the tree sequence data structure, the $\texttt{msprime}$ coalescent simulator, and our scalable inference framework: $\texttt{tsinfer}$. 

# Outline


1. [The tree sequence format](#ts)
2. [<span style="color: gray">Simulations (msprime & slim)</span>](#msprime)
3. [<span style="color: gray">Calculating statistics with tree sequences</span>](#stats)
4. [<span style="color: gray">Inferring tree sequences (tsinfer)</span>](ts_workshop_tsinfer.ipynb#tsinfer)
5. [<span style="color: gray">Metadata in tree sequences</span>](ts_workshop_tsinfer.ipynb#metadata)
6. [<span style="color: gray">References and Documentation</span>](ts_workshop_tsinfer.ipynb#ref)


In [ ]:
from IPython.display import SVG, display, HTML
import tskit

<a id='ts'></a>
# 1. The tree sequence format

A succint tree sequence is an efficient way of storing the ancestry of different parts of the genome. It can be used to store the output of evolutionary simulations, or to store an ancestry inferred from real data.

The primary benefits to this are:
1. Tree sequences provide an extremely efficient way to store *and* to access genetic data (i.e. they are [succinct data structures](https://en.wikipedia.org/wiki/Succinct_data_structure))

2. Tree sequences store ancestry rather than variation data (although the second can be trivially generated from the first). In other words, they reflect the evolutionary process rather than the genetic sequence which results from this process. Not only is this more meaningful, but for simulated tree sequences it also results in precision and efficiency gains, and for inferred tree sequences we hope it will lead to novel analytical approaches.

3.  There now exist efficient tools for evolutionary simulation and ancestral inference which produce tree sequences. In particular, these include the following
    -  Software for *simulating* tree sequences:
        -  Reverse time (coalescent) simulators: **`msprime`**
        -  Forward time simulators: **`SLiM`** (easiest), **`fwdpp`** (experimental, faster)
    -  Software for *inferring* tree sequences from real data
        -  **`tsinfer`**

4. There are built-in routines that simplify tree sequences to the minimum knowable ancestral history. This turns out to be a powerful operation.

## Terminology

A tree sequence is, as you might expect, a set of evolutionary trees, together with some associated data that specifies, for example, which branches in the trees contain individual mutations. Tree sequences can be accessed using the `tskit` python library.

### Nodes
Trees in a tree sequence are represented in a slightly unusual way. In the majority of libraries dealing with trees, each node is represented as an object in memory and the relationship between nodes as pointers between these objects. In tree sequences, however, all nodes, both internal and terminal, are represented by an integer ID, unique over the entire tree sequence.

The tips (leaves) of the tree, are usually marked as *sample nodes*: In most cases tips will correspond to samples with a full genetic sequence

Internal nodes represent ancestors in which a single DNA sequence was duplicated (in forwards-time terminology) or in which multiple sequences coalesced (in backwards-time terminology). 

### Edges
Edges specify a directional connection between node IDs in the tree. Since trees in a tree sequence are usually taken to represent marginal trees along a genome, as well as branches on the tree having a length (a time dimension), each branch also has a *span*, giving the length of a genome it covers.

### Genomes
A tree sequence is usually taken to cover a contiguous section of a single chromosome. For clarity of explanation, we use the word "genome" to refer to the section spanned by the whole tree sequence (if you find it easier, imagine that the examples in this notebook refer to to an organism with a single, short chromosome)

The tree below shows a pre-loaded example of a tree from a tree sequence

In [ ]:
# Ignore this code for the moment: it loads a tree sequence from an existing file and plots the 1st tree
my_tree_seq_object = tskit.load("data/tree_seq_file1.trees")
SVG(my_tree_seq_object.first().draw())

A genealogical tree like this one records the lines of descent along which genomes have been inherited. For example, ancestral genome 7 gave rise to two lineages which ended in sample genome 0 and sample genome 1 in the current population. 

#### A note on ploidy
Since diploids have two copies of each autosomal chromosome, diploid individuals contain two such lines of descent. The tree above might therefore representing three diploid (or 6 haploid) individuals. <aside>Note that tree sequences do have a concept of an individual that can contain multiple genomes (see [here](https://msprime.readthedocs.io/en/stable/interchange.html#nodes-genomes-or-individuals)), but currently this is only used when attaching metadata to a tree sequence (something detailed in part 3: [Metadata in tree sequences](ts_workshop_part3.ipynb#metadata)).</aside>

### Trees
The example above is a single tree. We might wish to inspect all the trees in a tree sequence. The simpest way to do this is to print them one after another. Our pre-loaded example was actually from a 1 kilobase tree sequence which happens to have 3 trees along the genome, one to the left, one in the middle of the DNA region, and one to the right.

In [ ]:
# Ignore all the code in this box for the moment - it simply draws pretty trees in a row
import itertools
def table(s):
    return "<table>{}</table>".format(s)
def tr(strs):
    return "".join(["<tr>{}</tr>".format(s) for s in strs])
def td(strs, align=["center"], width=[None], border=0, colspan=1):
    td_str = ""
    for s, a, w in zip(strs, itertools.cycle(align), itertools.cycle(width)):
        td_str += "<td{}{} style='padding:0; text-align: {}; border: {}'>{}</td>".format(
            "" if colspan==1 else" colspan='{}'".format(colspan), 
            "" if width is None else " width='{}'".format(w),
            a, border, s)
    return td_str

svg_trees=td(
    [tree.draw(width=150) for tree in my_tree_seq_object.trees()],
    width=[tree.span for tree in my_tree_seq_object.trees()],
    border="1px solid black")
rows = tr([svg_trees, 
           td([0, "", 1000], align=["left", "center", "right"]), 
           td(["genome position (bp)"], align=["center"], colspan=3)])

display(HTML(table(rows)))



Of course, the *sample nodes* 0..5 are present in all trees (i.e. at all genomic positions) since we have their full genomes. What you should note is that most of the *internal nodes* are also present in multiple trees: the exceptions are node 11 which is only present in the first tree, node 9 which is only present in the first two trees, and node 8 which is only present in the last tree.

However, in tree sequence terminology, we don't explictly keep track of where nodes start and end. Only edges (not nodes) possess a genomic span. So for example, this tree sequence is defined using edges like (0->7) which spans the entire genome, edges like (9->11) which only span the leftmost section of the genome, and edges like (10->12) which span the remain part of the genome. 

### Number of edges: a key tree sequence measure
Because edges can span multiple trees, they are an efficient way of storing ancestral information. In fact, the size of a tree sequence file is largely determined by the number of edges. This number can be used as a rough measure of many things such as the compressibility of a simulation, the speed with which analyses are likely to run, and even the effectiveness of ancestral inference. To give some sense of scale our inferred ancestry of human chromosome 20 in the UK Biobank dataset of a million genomes consists of 62 million edges which define 16 thousand trees; the uncompressed tree sequence takes 2.1 gigabytes of disk space, and on a reasonably specced computer it takes ~11 seconds to iterate over all these trees.

The `tskit` library allow us to use tree sequence objects in Python; these have specific properties and can also be queried using various methods, as documented in the [tskit Python API documentation](https://tskit.readthedocs.io/en/stable/python-api.html). For example, we can easily show the number of edges in a tree sequence as follows 

In [ ]:
my_tree_seq_object.num_edges # a lot less than the UK Biobank!

and we can iterate over the edges using the `.edges()` iterator, which returns an `Edge` object:

In [ ]:
for edge in my_tree_seq_object.edges():
  print(edge)

Similarly we can iterate over other components of a tree sequence, like the trees themselves. The `.trees()` iterator  returns a set of `Tree` objects, which we can `.draw()` in SVG or in unicode text format; this is how we produced the plots above.

In [ ]:
for tree in my_tree_seq_object.trees():
    display(HTML(tree.draw()))

Or we can access the tree at a genomic position, or the *nth* tree in the sequence.

In [ ]:
display(HTML("Tree at position 500 bp:" + my_tree_seq_object.at(500).draw()))
display(HTML("2nd tree in the sequence (index=1):" + my_tree_seq_object.at_index(1).draw()))

### Tree changes, ancestral recombinations, and SPRs

Note that adjacent trees in this tree sequence differ by a single "tree edit" or SPR (subtree-prune-and-regraft) operation: the subtree defined by node 6 in the first tree has been pruned and regrafted onto the edge from (12->7) to create the second tree. See if you can work out the SPR that converts the second tree to the third tree... 

In general, each detectable recombination occurring in ancestral history results in a single SPR in the tree sequence. If, as above, recombination breakpoints occurs at unique positions (an "infinite sites" model of breakpoints), then the number of trees in a tree sequence equals the number of ancestral recombination events minus one. If recombinations can occur at the same physical position (e.g. if a simulation is [run using a recombination map with finite loci](https://msprime.readthedocs.io/en/stable/api.html#msprime.RecombinationMap.uniform_map), then moving from one tree to the next in a tree sequence might require multiple SPRs, because there are multiple, overlaid ancestral recombination events.

If you are moving from tree to tree, but only need to work out what changed each time, you can use the `.edge_diffs()` method on the tree sequence. We will encounter this later, when calculating incremental statistics over a tree sequence.

## Tables: the underlying data store

The underlying data in tree sequences is stored as a set of tables. For example, there is a Nodes table and an Edges table, as well as others like Sites, Mutations, Populations, etc. The API for accessing the underlying tables of a tree sequence is documented [here](https://tskit.readthedocs.io/en/latest/data-model.html#sec-binary-interchange).

Accessing tree sequence data via tables is an alternative to using the built-in methods and properties of `TreeSequence` objects: it requires a deeper working knowledge of the tree sequence implementation, but it is more powerful. For example, it can be used to edit tree sequences, which are otherwise immutable objects. As the documentation says:  
<blockquote>To change the data held in a particular tree sequence, first get the table information as a <code>TableCollection</code> instance (using <code>dump_tables()</code>), edit those tables using the tables api, and create a new tree sequence using <code>TableCollection.tree_sequence()</code>.</blockquote>

In summary, since a tree sequence object is entirely specified by tables, these provide access to the guts of a stored tree sequence. Below is a simple example of inspecting all tables; later in this workshop we will meet examples where accessing the raw table data allows simple and fast extraction of statistics for plotting.

In [ ]:
print(my_tree_seq_object.tables)

## Tree sequences and ARGs
Much of the literature on full ancestral inference concentrates on the Ancestral Recombination Graph, or ARG. A tree sequence is essentially a compact representation of an ARG. The main difference is that ARGs store details of the position and potentially the timing of recombination events, whereas tree sequences simply store the trees created by these events. Many (but not all) details of the actual recombination events can be inferred from the changes in edges along a tree sequence. As we state in the [tsinfer paper](https://www.biorxiv.org/content/early/2018/11/01/458067)
<blockquote>an ARG encodes the events that occurred in the history of a sample, whereas a tree sequence encodes the outcome of those events</blockquote>

<a id='msprime'></a>
# 2. Simulations

## Reverse-time simulations with `msprime`
The `msprime` program is an efficient reimplementation of Hudson's $\texttt{ms}$ simulator which outputs coalescent trees using the tree sequence format. The easiest way to use it is through the Python API, which is well documented at https://msprime.readthedocs.io/en/stable/api.html. 

Running simulations is very straightforward, and produces trees with branch lengths as expected under the full coalescent.

In [ ]:
import msprime
import pyslim # to interface with SLiM files 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# simulate the coalescent for 6 samples, with effective population size of 10,000
# since the recombination rate defaults to zero, we should have only one tree
singular_tree_seq = msprime.simulate(sample_size=6, Ne=10000, random_seed=10)

# check there is only one tree in this ts
print("Number of Trees:",singular_tree_seq.num_trees)

#take the first (only) tree and visualize
first_tree=singular_tree_seq.first()
SVG(first_tree.draw())

Above we have simulated the coalescent for a sample of size six with an effective population size of 10000 diploids, and then printed out a depiction of the resulting tree. The `simulate()` function returns a standard `TreeSequence` object. Here there is only one tree because we have not provided a value for recombination_rate, which therefore defaults to zero. We can access the first (and only) tree in this sequence using `first()` method of a `TreeSequence` object.

Note that although it is common for the lowest numbered nodes to correspond to the samples (i.e. tips of the trees, here with IDs 0 to 5) and the internal nodes to take higher values (e.g. the root node with ID 10), this may not always be the case e.g. in complex forward simulations or with ancient samples. Sometimes you may wish to check which node IDs correspond to samples, as follows:

In [ ]:
singular_tree_seq.samples()


Since the simulation was a basic one, the 6 samples take the first 6 node IDs. We can easily trace our path back to the root from a particular sample (e.g. the sample genome with ID 3) for any tree using the `parent()` method:

In [ ]:
node_id = 3
while node_id != msprime.NULL_NODE:
  print("node {}: time = {}".format(node_id, first_tree.time(node_id)))
  node_id = first_tree.parent(node_id)

Here we have iterated up the tree starting at node 3, stopping when we reached the root. We know that a node is a root if its parent is `msprime.NULL_NODE`, which is a special reserved node. (The value of the null node is -1, but we recommend using the symbolic constant to make code more readable.) We also use the `time()` method to get the time for each node, which corresponds to the time in generations at which the coalescence event happened during the simulation. 

We can also obtain the length of a branch joining a node to its parent using the `branch_length()` method; the total branch length in the tree is also stored in the `total_branch_length` attribute: 

In [ ]:
node = 7
print("Branch length above node", node, "is", first_tree.branch_length(node))
print("Total branch length in tree is", first_tree.total_branch_length)

We can also do the standard tree traversals using the ``nodes`` iterator:

In [ ]:
display(SVG(first_tree.draw()))
print("Default:\t", list(first_tree.nodes()))
print("Preorder:\t", list(first_tree.nodes(order="preorder")))
print("Inorder:\t", list(first_tree.nodes(order="inorder")))
print("Postorder:\t", list(first_tree.nodes(order="postorder")))

### Recombination
Simulating the history of a single locus is a very useful, but we are most often interesting in simulating the history of our sample across large genomic regions under the influence of recombination. The msprime API is specifically designed to make this common requirement both easy and efficient. To model genomic sequences under the influence of recombination we have two parameters to the `simulate()` function. The length parameter specifies the length of the simulated sequence, and is a floating point number, so recombination (and mutation) can occur at any location along the sequence (the units are arbitrary). If length is not supplied, it is assumed to be 1.0. The recombination_rate parameter specifies the rate of crossing over per unit of length per generation, and is zero by default. See the [API Documentation](https://msprime.readthedocs.io/en/stable/api.html#variable-recombination-rates) for a discussion of the precise recombination model used, and how to implement variable recombination rates.

Here, we repeatedly simulate the trees across over a 10kb region with a recombination rate of 2×10−8 per base per generation, with a diploid effective population size of 1000, and stop when we get a sequence that has 4 trees.

In [ ]:
import random

random.seed(5)
simple_tree_seq = None
while True:
    seed = random.randint(1,1000000)
    print(seed)
    simple_tree_seq = msprime.simulate(
        sample_size=7, Ne=1000, length=1e4, recombination_rate=2e-8, random_seed=seed)
    if simple_tree_seq.num_trees == 3:
        break
display(HTML("".join([tree.draw() for tree in simple_tree_seq.trees()])))

In [ ]:
tc = simple_tree_seq.tables
tc.sites.add_row(100, '0')
tc.mutations.add_row(0, 11, '1')
tc.sites.add_row(200, '0')
tc.mutations.add_row(1, 8, '1')
tc.sites.add_row(5000, '0')
tc.mutations.add_row(2, 13, '1')

tc.sites.add_row(8000, '0')
tc.mutations.add_row(3, 12, '1')


tc.sort()
ts = tc.tree_sequence()
print(ts.genotype_matrix().T)
print(ts.tables.edges)
display(HTML("".join([tree.draw() for tree in ts.trees()])))


As before, we use the `trees()` method to iterate over the trees in the sequence. For each tree we print out its index (i.e. its position in the sequence) and the interval the tree covers (i.e. the genomic coordinates which all share precisely this tree) using the index and interval attributes. Thus, the first tree covers the first 2kb of sequence, the second tree covers the remaining 8kb.

Note that this is a case where, although there has been a recombination event in the history of the samples, it has not changed the *topology* of the tree. Instead, it has changed the depth of one of the coalescent points: essentially, the subtree defined by node 10 has been pruned, and grafted back onto the same branch, a little lower down.

### Mutations
Mutations are generated in msprime by throwing mutations down on the branches of trees at a particular rate. The mutations are generated under the infinite sites model, and so each mutation occurs at a unique (floating point) point position along the genomic interval occupied by a tree. The mutation rate for simulations is specified using the mutation_rate parameter of `simulate()`. For example, the following chunk simulates 50kb of nonrecombining sequence with a mutation rate of 1×10−8 per base per generation:

In [ ]:
mut_tree_seq = msprime.simulate(
  sample_size=6, Ne=1000, length=50e3, mutation_rate=1e-8, random_seed=30)
print("Num trees:", mut_tree_seq.num_trees)
tree = mut_tree_seq.first()
for site in tree.sites():
    for mutation in site.mutations:
        print("Mutation {} @ position {:.2f} over node {}".format(
        mutation.id, site.position, mutation.node))
display(SVG(tree.draw()))

Mutations in the tree sequence lead to variable sites. By default, `msprime` creates mutations where the original (ancestral) state is `0` and the derived state is `1`. For example, in the above example, only samples 0 and 4 inherit the mutations *1* and *2* (which are at positions 4485.17 and 9788.56 along the sequence), meaning that they both have the variant `1` at those positions, whereas the other samples have the variant `0` at those positions.

It should be clear that the mutations plus tree are enough to reconstruct the state of the variant sites for all samples. Thus the full tree sequence structure (which includes all mutations) encodes the full variant matrix. Often we do not need to actually deal with the variant matrix; nevertheless we will see later how to take a tree sequence and save the variant matrix as a VCF file, or extract the distribution of a specific variant into a numpy array for further analysis.

### Population structure in `msprime`
`Msprime` provides a flexible and simple way to model past demographic events in arbitrary combinations. Here is an example, taken from the [`msprime` documentation](https://msprime.readthedocs.io/en/stable/tutorial.html#demography), describing the Gutenkunst et al. out-of-Africa model (2009). Figure 2B is a schematic of this model, reproduced below. See [Table 1](https://journals.plos.org/plosgenetics/article/figure?id=10.1371/journal.pgen.1000695.t001)  for the values used. Coalescent simulation moves from the present back into the past, so times are in units of generations ago, and we build the model with most recent events first.

The following is figure 2b from the paper:

<img style="width: 400px;" src="images/gutenkunst_cropped_fig2b.png">

In [ ]:
import math
def out_of_africa():
    # First we set out the maximum likelihood values of the various parameters
    # given in Table 1.
    N_A = 7300
    N_B = 2100
 
    N_AF = 12300
    N_EU0 = 1000
    N_AS0 = 510
    
    # Times are provided in years, so we convert into generations.
    generation_time = 25
    T_AF = 220e3 / generation_time
    T_B = 140e3 / generation_time
    T_EU_AS = 21.2e3 / generation_time
    # We need to work out the starting (diploid) population sizes based on
    # the growth rates provided for these two populations
    r_EU = 0.004
    r_AS = 0.0055
    N_EU = N_EU0 / math.exp(-r_EU * T_EU_AS)
    N_AS = N_AS0 / math.exp(-r_AS * T_EU_AS)
    # Migration rates during the various epochs.
    m_AF_B = 25e-5
    m_AF_EU = 3e-5
    m_AF_AS = 1.9e-5
    m_EU_AS = 9.6e-5
    # Population IDs correspond to their indexes in the population
    # configuration array. Therefore, we have 0=YRI, 1=CEU and 2=CHB
    # initially.
    model = dict(
        #we simulate two chromosomes from each population
        population_configurations = [
            msprime.PopulationConfiguration(
                sample_size=2, initial_size=N_AF),
            msprime.PopulationConfiguration(
                sample_size=2, initial_size=N_EU, growth_rate=r_EU),
            msprime.PopulationConfiguration(
                sample_size=2, initial_size=N_AS, growth_rate=r_AS)
        ],
        migration_matrix = [
            [      0, m_AF_EU, m_AF_AS],
            [m_AF_EU,       0, m_EU_AS],
            [m_AF_AS, m_EU_AS,       0],
        ],
        demographic_events = [
            # CEU and CHB merge into B with rate changes at T_EU_AS
            msprime.MassMigration(
                time=T_EU_AS, source=2, destination=1, proportion=1.0),
            msprime.MigrationRateChange(time=T_EU_AS, rate=0),
            msprime.MigrationRateChange(
                time=T_EU_AS, rate=m_AF_B, matrix_index=(0, 1)),
            msprime.MigrationRateChange(
                time=T_EU_AS, rate=m_AF_B, matrix_index=(1, 0)),
            msprime.PopulationParametersChange(
                time=T_EU_AS, initial_size=N_B, growth_rate=0, population_id=1),
            # Population B merges into YRI at T_B
            msprime.MassMigration(
                time=T_B, source=1, destination=0, proportion=1.0),
            # Size changes to N_A at T_AF
            msprime.PopulationParametersChange(
                time=T_AF, initial_size=N_A, population_id=0)
        ]
    )
    # Use the demography debugger to print out the demographic history
    # that we have just described.
    dd = msprime.DemographyDebugger(
        population_configurations=model["population_configurations"],
        migration_matrix=model["migration_matrix"],
        demographic_events=model["demographic_events"])
    dd.print_history()
    return model

The DemographyDebugger provides a method to debug the history that you have described so that you can be sure that the migration rates, population sizes and growth rates are all as you intend during each epoch:

In [ ]:
model = out_of_africa()
OOA_tree_seq = msprime.simulate(**model,
                              length=5e3, 
                              recombination_rate=1e-8, 
                              random_seed=11)

Let's look at the output from running the simulation with the following parameters:

In [ ]:
for tree in OOA_tree_seq.trees():
    display(SVG(tree.draw(mutation_labels={},
        node_labels={0:"AFR1",1:"AFR2", 2:"EUR1",3:"EUR2",4:"ASN1",5:"ASN2"},
        width=500)))

### Saving files
Let's throw some mutations onto this genealogy, then save the resulting tree sequence file. We can also save the variation data into a VCF file, which we will use when demonstrating `tsinfer` in section 4. 

In [ ]:
# Add mutations to an existing tree sequence
OOA_with_mutations = msprime.mutate(OOA_tree_seq, rate=1e-6, random_seed=1, keep=True)

# Save the tree sequence file (recommended suffix is .trees)
OOA_with_mutations.dump("data/OOA.trees")

# Save as vcf
with open("data/3pop_simulation.vcf", "w") as vcf_file:
    OOA_with_mutations.write_vcf(vcf_file, ploidy=2)

## Forward Simulations with `SLiM`
Although msprime is very effective for producing simulations under the coalescent, there are many evolutionary processes that are hard to simulate using coalescent methodology (e.g. models with natural selection, complex demography, or complex life histories). Recently a fast forwards simulator called [SLiM](https://messerlab.org/slim/) (Selection on Linked Mutations) has been developed that can save tree sequences as output.


---

SLiM is fast yet extremely flexible. Simulations are specified using a simple R-like language called `Eidos`: there is a GUI (Mac OS X only) which can help considerably in debugging and visualising simulations. However, for portability, here we simply save the Eidos commands into a temporary file and give that file to the `slim` executable in Python. Using python to do this allows us to manipulate the tree sequence output after the simulation is done. 

<aside>By default, SLiM does not save tree sequences: this functionality is activated by calling `initializeTreeSeq();` when initializing the simulation</aside>

### A simple model

The Eidos script embedded into Python below is taken from recipe 16.3 of the  [extensive SLiM manual](http://benhaller.com/slim/SLiM_Manual.pdf), with a few additional comments (`//double slashes distinguish comments in Eidos`). We will not go into many details of the Eidos language: it is well covered in the manual. Nevertheless, for anyone familiar with genetic simulations in R or Python, it should be reasonably obvious what is going on in the eidos_script defined below. By default, SLiM runs a standard Wright-Fisher with panmixis and discrete generations. Top-level blocks delimited by `{}` are each run at a different generation (or range of generations). As in C, code needs to be semicolon-terminated.

The script simulates genomes with a background level of mildly deleterious mutations, and introduces a single advantageous mutation after a certain number of burn in generations, tracking it until it is lost (in which case we restart the simulation from when it was introduced) or is fixed (in which case we save the output to a tree sequence). Although it is fast as forward simulations go, it is still much slower than coalescent simulators, especially for large population sizes (e.g. here we only simulate a population of 500 diploids).

In [ ]:
import subprocess
import string
import tempfile

# NB: use string.Template to pass parameters from Python into the Eidos script
# These python-substituted parameters are written in CAPS
eidos_script = string.Template(r"""
initialize() {
    initializeTreeSeq();
    initializeMutationRate($MU_DEL);
    initializeMutationType("m2", $H_DEL, "g", $S_DEL, 1.0);  // deleterious
    initializeMutationType("m3", $H_ADV, "f", $S_ADV);       // advantageous
    // only deleterious mutations (type m2) should be introduced automatically
    initializeGenomicElementType("g1", m2, 1.0);
    initializeGenomicElement(g1, 0, $LENGTH);
    initializeRecombinationRate($RHO);
}

1 { // in the first generation, add a standard population of $POPSIZE diploids
    defineConstant("simID", getSeed());
    sim.addSubpop("p1", $POPSIZE);
}

$BURN_IN_GENS late() { // after $BURN_IN_GENS introduce an advantageous mutation
    target = sample(p1.genomes, 1);
    target.addNewDrawnMutation(m3, $SELECTED_VARIANT_POS);
    sim.treeSeqOutput("data/slim_init.trees"); // save in case restart needed
}

$BURN_IN_GENS:$MAX_GENS late() { // check every subsequent generation for fixation or loss
    if (sim.countOfMutationsOfType(m3) == 0) {
        if (sum(sim.substitutions.mutationType == m3) == 1) {
            fn = "data/slim_fixed.trees";
            sim.treeSeqOutput(fn);
            catn(simID + ": fixed and saved to '" + fn + "'");
            sim.simulationFinished();
        } else {
            catn(simID + ": lost - restarting");
            sim.readFromPopulationFile("data/slim_init.trees");
            setSeed(getSeed() + 1); // try again with a different seed
        }
    }
}""")
random_seed = 123
sim_script = eidos_script.substitute(
    H_DEL = 0.5,   # deleterious dominance coeffficient,
    H_ADV = 1.0,   # advantageous dominance_coef
    S_DEL = -0.01, # deleterious selection_coefficient,
    S_ADV = 0.05,  # advantageous selection_coefficient,
    MU_DEL = 1e-8, # per base per generation mutation rate for deleterious muts
    RHO= 1e-7,     # recombination rate,
    POPSIZE = 500,
    LENGTH = 99999, # chromosome length
    SELECTED_VARIANT_POS = 50000, # chromosome length
    BURN_IN_GENS = 1000,
    MAX_GENS = 100000
    )
with tempfile.NamedTemporaryFile(mode='w+t') as script:
    script.write(sim_script)
    script.flush()
    process = subprocess.Popen(['slim','-seed',str(random_seed), script.name],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, 
        universal_newlines=True)
    print("Output from SLiM follows:")
    for line in iter(process.stdout.readline, ''):
        print("SLiM> " + line, end="")

### Adding neutral mutations
The SLiM simulation should have resulted in population with a fixed advantageous mutation, and generated a tree sequence file for further use. The main reason that the simulation is relatively fast is that we have not had to track neutral mutations during the run. Since (by definition), neutral mutations do not affect the simulation, we can add any number of them after the simulation has finished, as we did with the out of Africa simulation in msprime, then dump the tree sequence out to a `.trees` file:

In [ ]:
import pyslim # To load ts files from SLiM use pyslim, not `msprime` or `tskit`

# NB for efficiency, we could .simplify() this tree seq, as in the SLiM manual
ts_fwd = pyslim.load("data/slim_fixed.trees")
print("Number of (selected) mutations at end of forward simulation:", ts_fwd.num_mutations)
ts_fwd_mut = msprime.mutate(ts_fwd, rate=1e-7, random_seed=1, keep=True)
print("Number of mutations after adding neutral variation:", ts_fwd_mut.num_mutations)
ts_fwd_mut.dump("data/slim_fixed_with_neutral.trees")

Note that an important property of forwards time trees is that not all lineages are guaranteed to have coalesced, so they may contain **multiple roots**. Thus more than one tree-like structure may exist for each "tree" in the tree sequence, as shown below:

In [ ]:
ts_sub = ts_fwd.simplify(ts_fwd.samples()[:10])
for tree in ts_sub.trees():
    display(SVG(tree.draw()))
    if tree.index == 1:
        break # only draw the first 2 "trees"

<a id='stats'></a>
# 3. Calculations on Tree Sequences
The tree sequence format can be seen as an "evolutionary encoding" of DNA sequence data: although it is often not needed, the raw sequence (or variant) data can be easily extracted from a tree sequence. The tree sequence also allows efficient calculation of statistics of interest such as pairwise diversity statistics and recombination maps. 

### Extracting sequence data or variants
We might be interested in accessing the sequence data that results from simulations directly. The most efficient way to do this is by using the `TreeSequence.variants()` method, which returns an iterator over all the Variant objects arising from the trees and mutations. Each variant contains a reference to the site object, as well as the alleles and the observed sequences for each sample in the genotypes field. As a demonstration, we do this for the simple mutated tree sequence we calculated previously, that only contains one tree.

In [ ]:
display(SVG(mut_tree_seq.first().draw()))
for variant in mut_tree_seq.variants():
    print(
        "Variant", variant.site.id, variant.site.position,
        variant.alleles, variant.genotypes, sep="\t")

Here we have looped through each variant, printing out the observed state of each sample as an array of zeros and ones, along with the index and position of the corresponding mutation. In this example, the alleles are always '0' (the ancestral state) and '1' (the derived state), because we are simulating with the infinite sites mutation model, in which each mutation occurs at a unique position in the genome. More complex models are possible, however.

This way of working with the sequence data is quite efficient because we do not need to keep the entire genotype matrix in memory at once. However, if we do want the full genotype matrix it is simple to obtain:

In [ ]:
A = mut_tree_seq.genotype_matrix()
A

## Integration with `scikit-allel`

Because tskit integrates tightly with numpy, we can export data into a number of upstream toolkits very efficiently. In particular, we can export data into [scikit allel](scikit-allel.readthedocs.io/en/latest/) very easily:

In [ ]:
# Run a simple simulation
ts = msprime.simulate(
    sample_size=1000, Ne=10000, length=1e6, mutation_rate=1e-8,
    recombination_rate=1e-8,random_seed=10)


In [ ]:
ts.num_sites, ts.num_samples

In [ ]:
import allel

# Export the genotype data to allel. Unfortunately there's a slight mismatch in the 
# terminology here where genotypes and haplotypes mean different things in the two
# libraries.
h = allel.HaplotypeArray(ts.genotype_matrix())
print(h.n_variants, h.n_haplotypes)
h

Sckit.allel has a wide-ranging and efficient suite of tools for working with genotype data, so should provide anything that's needed. For example, we can compute pairwise diversity, $\pi$.

In [ ]:
ac = h.count_alleles()
v = allel.stats.mean_pairwise_difference(ac)
v

Tskit also supports computing statistics directly from the trees, and can be much faster than alternatives for large tree sequences, especially if (as we shall soon see) the statistic can be calculated incrementally. Only a handful of statistics are currently supported, but many will be implemented soon.

In [ ]:
ts.pairwise_diversity(), np.sum(v)

## LD Plot
The msprime API provides methods to efficiently calculate population genetics statistics. For example, the LdCalculator class allows us to compute pairwise linkage disequilibrium coefficients. Here we use the get_r2_matrix() method to easily make an LD plot using matplotlib. (Thanks to the excellent scikit-allel for the basic plotting code used here.)

In [ ]:

ts = msprime.simulate(100, recombination_rate=10, mutation_rate=20, random_seed=1)
ld_calc = msprime.LdCalculator(ts)
A = ld_calc.r2_matrix()
# Now plot this matrix.
x = A.shape[0] / plt.rcParams['figure.dpi']
x = max(x, plt.rcParams['figure.figsize'][0])
fig, ax = plt.subplots(figsize=(x, x))
fig.tight_layout(pad=0)
im = ax.imshow(A, interpolation="none", vmin=0, vmax=1, cmap="Blues")
ax.set_xticks([])
ax.set_yticks([])
for s in 'top', 'bottom', 'left', 'right':
    ax.spines[s].set_visible(False)
plt.gcf().colorbar(im, shrink=.5, pad=0);     
    


## Very large datasets

One of the big advantages of msprime/tskit is that we can simulate and process genetic data on a very large scale. For example, here is a simulation of 100K human-like 10Mb chromosomes:

In [ ]:
%%time
ts = msprime.simulate(
    sample_size=10**5, length=1e7, recombination_rate=1e-8, Ne=10**4, random_seed=42)

In [ ]:
ts.num_trees

In [ ]:
# Write the tree sequence to file.
ts.dump("data/large-ts-no-mutations.trees")

# Get the size
import os
print("File size = {}MiB".format(os.path.getsize("data/large-ts-no-mutations.trees") / 2**20))

In [ ]:
%%time
# Loading is fast
ts = msprime.load("data/large-ts-no-mutations.trees")

In [ ]:
%%time
# So is iterating over all trees
total = 0  
for tree in ts.trees():
    total += tree.interval[1] - tree.interval[0] # Just to do *something* in the loop

print(total, ts.sequence_length)

In [ ]:
%%time
# Insert a ridiculous number of mutations
mutated_ts = msprime.mutate(ts, rate=1e-6, random_seed=42) # deliberately high mutation rate

In [ ]:
print("We just created {:,} variant sites!".format(mutated_ts.num_sites))

In [ ]:
mutated_ts.dump("data/large-ts-mutations.trees")

print("Tree sequence size = {:.2f}MiB".format(os.path.getsize("data/large-ts-mutations.trees") / 2**20))

In [ ]:
print("Genotype matrix size: {:.2f}GiB".format(
    mutated_ts.num_sites * mutated_ts.num_samples / 2**30))

In [ ]:
%%time
mutated_ts.pairwise_diversity()

## Incremental calculations

The tree sequence data structure is efficient because it captures the shared structure between adjacent trees. Understanding how trees change as we move across the sequence is also critical for defining efficient algorithms.

Consider the problem of computing the total branch length. In a single tree, all we can do is to iterate over all the nodes and sum the length of each branch. If we repeat this for every tree in the sequence, this ends up being a slow operation:

In [ ]:
%%time
total_branch_length = np.zeros(ts.num_trees)
for tree in ts.trees():
    total_branch_length[tree.index] = tree.total_branch_length
    if tree.index == 50:
        break

print(total_branch_length[:5])

Here it takes tens of seconds to compute the total branch length for the first 50 trees --- these are large trees with 200K nodes! Since there are about 1000 times more trees than this, it will take a long time to compute the total branch length for all trees.

However, we can do much better than this. As we noted in the introductory section, the [`edge_diffs()`](https://msprime.readthedocs.io/en/stable/api.html#msprime.TreeSequence.edge_diffs) method returns an iterator over the edges that are inserted and removed for each tree in turn. Trees don't change very much as we move across the tree sequence; in fact, for coalescent trees, we know that there are at most **four** edges that are removed and inserted at each tree transition. By taking this information into account, we can compute the total branch length for all trees in a tiny fraction of the time:

In [ ]:
%%time
current = 0
total_branch_length = np.zeros(ts.num_trees)
node_time = ts.tables.nodes.time
for j, (_, edges_out, edges_in) in enumerate(ts.edge_diffs()):
    for e in edges_out:
        current -= node_time[e.parent] - node_time[e.child]
    for e in edges_in:
        current += node_time[e.parent] - node_time[e.child]
    total_branch_length[j] = current

print(total_branch_length[:5])

In [ ]:
plt.plot(total_branch_length);

## Tables API

The tables API provides a very efficient way of getting at the underlying numerical data representing the tree sequence. Suppose we wished to plot the distribution of the length of the edges in the simulation (an edge is a (left, right, parent, child) tuple describing a branch in the trees spanning the interval from left to right).

See the [documentation](https://msprime.readthedocs.io/en/stable/api.html#tables-api) on this API.

In [ ]:
%%time
tables = mutated_ts.tables
edge_length = tables.edges.right - tables.edges.left

In [ ]:
plt.hist(edge_length);

We can also use this to access information about the sites and mutations (which are in their own respective tables --- see the [data model](https://msprime.readthedocs.io/en/stable/interchange.html#sec-data-model) for details). Here, for example, we get an array of the time of each mutation (or, more accurately, the time of the node over which each mutation occured), find how many of these are old and plot their age distribution.

In [ ]:
oldest_time = np.max(tables.nodes.time)
mutation_time = tables.nodes.time[tables.mutations.node]
index = mutation_time > (oldest_time / 10)
print("number of mutations older than 10% of the oldest time = ", np.sum(index))

In [ ]:
plt.hist(mutation_time[index]);